# Неделя 1. Подготовка данных к анализу и построению моделей

Первая часть проекта посвящена подготовке данных для дальнейшего описательного анализа и построения прогнозных моделей. Надо будет написать код для предобработки данных (исходно посещенные веб-сайты указаны для каждого пользователя в отдельном файле) и формирования единой обучающей выборки. Также в этой части мы познакомимся с разреженным форматом данных (матрицы Scipy.sparse), который хорошо подходит для данной задачи. 

**План 1 недели:**
 - Часть 1. Подготовка обучающей выборки
 - Часть 2. Работа с разреженным форматом данных

Наконец, для лучшей воспроизводимости результатов приведем список версий основных используемых в проекте библиотек: NumPy, SciPy, Pandas, Matplotlib, Statsmodels и Scikit-learn. Для этого воспользуемся расширением [watermark](https://github.com/rasbt/watermark).

In [1]:
%load_ext watermark

In [2]:
%watermark -v -m -p numpy,scipy,pandas,matplotlib,statsmodels,sklearn -g

CPython 3.7.3
IPython 7.6.1

numpy 1.16.4
scipy 1.2.1
pandas 0.24.2
matplotlib 3.1.0
statsmodels 0.10.0
sklearn 0.0

compiler   : MSC v.1915 64 bit (AMD64)
system     : Windows
release    : 10
machine    : AMD64
processor  : Intel64 Family 6 Model 142 Stepping 9, GenuineIntel
CPU cores  : 4
interpreter: 64bit
Git hash   : f71f8b5705d84722e0a6785677a271ac19bc899e


In [3]:
from __future__ import division, print_function

import warnings
warnings.filterwarnings('ignore')

from glob import glob
import pickle
from tqdm import tqdm 

import os
from os import listdir
from os.path import isfile, join


import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix

**Посмотрим на один из файлов с данными о посещенных пользователем (номер 31) веб-страницах.**

In [4]:
user31_data = pd.read_csv('10users/user0031.csv')

In [5]:
user31_data.head()

,timestamp,site
0,2013-11-15 08:12:07,fpdownload2.macromedia.com
1,2013-11-15 08:12:17,laposte.net
2,2013-11-15 08:12:17,www.laposte.net
3,2013-11-15 08:12:17,www.google.com
4,2013-11-15 08:12:18,www.laposte.net


**Поставим задачу классификации: идентифицировать пользователя по сессии из 10 подряд посещенных сайтов. Объектом в этой задаче будет сессия из 10 сайтов, последовательно посещенных одним и тем же пользователем, признаками – индексы этих 10 сайтов. Целевым классом будет id пользователя.**

### <center>Пример для иллюстрации</center>
**Пусть пользователя всего 2, длина сессии – 2 сайта.**

<center>user0001.csv</center>
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-yw4l{vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-031e">timestamp</th>
    <th class="tg-031e">site</th>
  </tr>
  <tr>
    <td class="tg-031e">00:00:01</td>
    <td class="tg-031e">vk.com</td>
  </tr>
  <tr>
    <td class="tg-yw4l">00:00:11</td>
    <td class="tg-yw4l">google.com</td>
  </tr>
  <tr>
    <td class="tg-031e">00:00:16</td>
    <td class="tg-031e">vk.com</td>
  </tr>
  <tr>
    <td class="tg-031e">00:00:20</td>
    <td class="tg-031e">yandex.ru</td>
  </tr>
</table>

<center>user0002.csv</center>
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-yw4l{vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-031e">timestamp</th>
    <th class="tg-031e">site</th>
  </tr>
  <tr>
    <td class="tg-031e">00:00:02</td>
    <td class="tg-031e">yandex.ru</td>
  </tr>
  <tr>
    <td class="tg-yw4l">00:00:14</td>
    <td class="tg-yw4l">google.com</td>
  </tr>
  <tr>
    <td class="tg-031e">00:00:17</td>
    <td class="tg-031e">facebook.com</td>
  </tr>
  <tr>
    <td class="tg-031e">00:00:25</td>
    <td class="tg-031e">yandex.ru</td>
  </tr>
</table>

Идем по 1 файлу, нумеруем сайты подряд: vk.com – 1, google.com – 2 и т.д. Далее по второму файлу. 

Отображение сайтов в их индесы должно получиться таким:

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-yw4l{vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-031e">site</th>
    <th class="tg-yw4l">site_id</th>
  </tr>
  <tr>
    <td class="tg-yw4l">vk.com</td>
    <td class="tg-yw4l">1</td>
  </tr>
  <tr>
    <td class="tg-yw4l">google.com</td>
    <td class="tg-yw4l">2</td>
  </tr>
  <tr>
    <td class="tg-yw4l">yandex.ru</td>
    <td class="tg-yw4l">3</td>
  </tr>
  <tr>
    <td class="tg-yw4l">facebook.com</td>
    <td class="tg-yw4l">4</td>
  </tr>
</table>

Тогда обучающая выборка будет такой (целевой признак – user_id):
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-s6z2{text-align:center}
.tg .tg-baqh{text-align:center;vertical-align:top}
.tg .tg-hgcj{font-weight:bold;text-align:center}
.tg .tg-amwm{font-weight:bold;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-hgcj">session_id</th>
    <th class="tg-hgcj">site1</th>
    <th class="tg-hgcj">site2</th>
    <th class="tg-amwm">user_id</th>
  </tr>
  <tr>
    <td class="tg-s6z2">1</td>
    <td class="tg-s6z2">1</td>
    <td class="tg-s6z2">2</td>
    <td class="tg-baqh">1</td>
  </tr>
  <tr>
    <td class="tg-s6z2">2</td>
    <td class="tg-s6z2">1</td>
    <td class="tg-s6z2">3</td>
    <td class="tg-baqh">1</td>
  </tr>
  <tr>
    <td class="tg-s6z2">3</td>
    <td class="tg-s6z2">3</td>
    <td class="tg-s6z2">2</td>
    <td class="tg-baqh">2</td>
  </tr>
  <tr>
    <td class="tg-s6z2">4</td>
    <td class="tg-s6z2">4</td>
    <td class="tg-s6z2">3</td>
    <td class="tg-baqh">2</td>
  </tr>
</table>

Здесь 1 объект – это сессия из 2 посещенных сайтов 1-ым пользователем (target=1). Это сайты vk.com и google.com (номер 1 и 2). И так далее, всего 4 сессии. Пока сессии у нас не пересекаются по сайтам, то есть посещение каждого отдельного сайта относится только к одной сессии.

## Часть 1. Подготовка обучающей выборки
Реализуйте функцию *prepare_train_set*, которая принимает на вход путь к каталогу с csv-файлами *path_to_csv_files* и параметр *session_length* – длину сессии, а возвращает 2 объекта:
- DataFrame, в котором строки соответствуют уникальным сессиям из *session_length* сайтов, *session_length* столбцов – индексам этих *session_length* сайтов и последний столбец – ID пользователя
- частотный словарь сайтов вида {'site_string': [site_id, site_freq]}, например для недавнего игрушечного примера это будет {'vk.com': (1, 2), 'google.com': (2, 2), 'yandex.ru': (3, 3), 'facebook.com': (4, 1)}

In [6]:
def prepare_train_set(path_to_csv_files, session_length=10):
    files = glob(path_to_csv_files + "/*.csv")
    
    files_list = []
    for filename in files:
        file_data = pd.read_csv(filename, index_col=None, header=0)
        files_list.append(file_data)

    sites = pd.concat(files_list, axis=0, ignore_index=True).iloc[:, 1]

    dict_freq = {}
    ind = 1
    
    for site in sorted(sites):
        if site not in dict_freq:
            dict_freq[site] = [ind, 1]
            ind += 1
        else:
            dict_freq[site][1] += 1
            
            
    sessions = []
    for i, file in enumerate(files):
        user_session = pd.read_csv(file, index_col=None, header=0).iloc[:, 1] 
        cur_session = []
        
        for j, session in enumerate(user_session):
            
            cur_session.append(dict_freq[session][0])
            
            if j == len(user_session) - 1:
                while len(cur_session) % 10 != 0: cur_session.append(0)
            
            if len(cur_session) % 10 == 0 and len(cur_session):    
                sessions.append(cur_session + [i + 1])
                cur_session = []
            
    train_set = pd.DataFrame(sessions)
    train_set.columns = [f'site{i}' for i in range(1, session_length + 1)] + ['user_id']
    
    return train_set, dict_freq

In [7]:
train_data_3users, site_freq_3users = prepare_train_set('3users')

In [8]:
train_data_3users

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,user_id
0,10,8,8,4,8,5,1,6,2,9,1
1,10,5,5,5,0,0,0,0,0,0,1
2,10,8,3,3,8,0,0,0,0,0,2
3,7,5,8,5,8,5,5,6,11,7,3
4,7,5,8,0,0,0,0,0,0,0,3


In [9]:
site_freq_3users

{'accounts.google.com': [1, 1],
 'apis.google.com': [2, 1],
 'football.kulichki.ru': [3, 2],
 'geo.mozilla.org': [4, 1],
 'google.com': [5, 9],
 'mail.google.com': [6, 2],
 'meduza.io': [7, 3],
 'oracle.com': [8, 8],
 'plus.google.com': [9, 1],
 'vk.com': [10, 3],
 'yandex.ru': [11, 1]}

**1. Примените полученную функцию к данным по 10 пользователям, посмотрите, сколько уникальных сессий из 10 сайтов получится, запишите ответ в файл с помощью функции *write_answer_to_file* – это будет ответом к первому вопросу теста. Если ответ получился неправильный, значит где-то ошибка в функции *prepare_train_set*.**

In [10]:
train_data_10users, site_freq_10users = prepare_train_set('10users')

In [11]:
def write_answer_to_file(answer, file_address):
    with open(file_address, 'w') as out_f:
        out_f.write(str(answer))

In [12]:
write_answer_to_file(len(train_data_10users), 'answer1_1.txt')

**2. Сколько всего уникальных сайтов в выборке из 10 пользователей? **

In [13]:
write_answer_to_file(len(site_freq_10users), 'answer1_2.txt')

**3. Примените полученную функцию к данным по 150 пользователям, посмотрите, сколько уникальных сессий из 150 сайтов получится, запишите ответ в файл с помощью функции *write_answer_to_file* – это будет ответом к третьему вопросу. Если ответ получился неправильный, значит где-то ошибка в функции *prepare_train_set*.**

In [14]:
%%time
train_data_150users, site_freq_150users = prepare_train_set('150users')

Wall time: 16.4 s


In [15]:
write_answer_to_file(len(train_data_150users), 'answer1_3.txt')

**4. Сколько всего уникальных сайтов в выборке из 150 пользователей? **

In [16]:
write_answer_to_file(len(site_freq_150users), 'answer1_4.txt')

**5. Выведите топ-10 самых популярных сайтов среди посещенных 150 пользователями. Запишите ответ в файл *answer1_5.txt* через пробел – все 10 сайтов на одной строке.**

In [17]:
site_pop = {site: site_freq_150users[site][1] for site in site_freq_150users}

top10_popular = []
for i, site in enumerate(sorted(site_pop, key=site_pop.get, reverse=True)):
    if i == 10: break
    top10_popular.append(site)

In [18]:
top10_popular

['www.google.fr',
 'www.google.com',
 'www.facebook.com',
 'apis.google.com',
 's.youtube.com',
 'clients1.google.com',
 'mail.google.com',
 'plus.google.com',
 'safebrowsing-cache.google.com',
 'www.youtube.com']

In [19]:
write_answer_to_file(" ".join(str(site) for site in top10_popular), 'answer1_5.txt')

**Для дальнейшего анализа запишем полученные объекты DataFrame в csv-файлы.**

In [20]:
train_data_10users.to_csv('train_data_10users.csv', 
                          index_label='session_id', 
                          float_format='%d')

train_data_150users.to_csv('train_data_150users.csv', 
                         index_label='session_id', 
                           float_format='%d')

## Часть 2. Работа с разреженным форматом данных

** Если так подумать, то полученные признаки *site1*, ..., *site10* смысла не имеют как признаки в задаче классификации. А вот если воспользоваться идеей мешка слов из анализа текстов – это другое дело. Создадим новые матрицы, в которых строкам будут соответствовать сессии из 10 сайтов, а столбцам – индексы сайтов. На пересечении строки $i$ и столбца $j$ будет стоять число $n_{ij}$ – cколько раз сайт $j$ встретился в сессии номер $i$. Делать это будем с помощью разреженных матриц Scipy – [csr_matrix](https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.sparse.csr_matrix.html). Прочитайте документацию, разберитесь, как использовать разреженные матрицы и создайте такие матрицы для наших данных. **

In [86]:
X_10users, y_10users = train_data_10users.iloc[:, :-1].values, train_data_10users.iloc[:, -1].values
X_150users, y_150users = train_data_150users.iloc[:, :-1].values, train_data_150users.iloc[:, -1].values

In [87]:
indptr = [0]
indices = []
data = []
vocabulary = {}
for d in X_10users:
    for term in d:
        index = vocabulary.setdefault(term, len(vocabulary))
        indices.append(index)
        data.append(1)
    indptr.append(len(indices))

X_sparse_10users = csr_matrix((data, indices, indptr), dtype=np.int32).toarray()[:, 1:]

In [88]:
indptr = [0]
indices = []
data = []
vocabulary = {}
for d in X_150users:
    for term in d:
        index = vocabulary.setdefault(term, term)
        indices.append(index)
        data.append(1)
    indptr.append(len(indices))

X_sparse_150users = csr_matrix((data, indices, indptr), dtype=np.int8).toarray()[:, 1:]

**Сохраним эти разреженные матрицы с помощью [pickle](https://docs.python.org/2/library/pickle.html) (сериализация в Python), также сохраним вектора *y_10users, y_150users* – целевые значения (id пользователя)  в выборках из 10 и 150 пользователей. То что названия этих матриц начинаются с X и y, намекает на то, что на этих данных мы будем проверять первые модели классификации.
Наконец, сохраним также и частотные словари сайтов для 3, 10 и 150 пользователей.**

In [89]:
PATH_TO_DATA = ''

In [90]:
with open(os.path.join(PATH_TO_DATA, 
                       'X_sparse_10users.pkl'), 'wb') as X10_pkl:
    pickle.dump(X_sparse_10users, X10_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 
                       'y_10users.pkl'), 'wb') as y10_pkl:
    pickle.dump(y_10users, y10_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 
                       'X_sparse_150users.pkl'), 'wb') as X150_pkl:
    pickle.dump(X_sparse_150users, X150_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 
                       'y_150users.pkl'), 'wb') as y150_pkl:
    pickle.dump(y_150users, y150_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 
                       'site_freq_3users.pkl'), 'wb') as site_freq_3users_pkl:
    pickle.dump(site_freq_3users, site_freq_3users_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 
                       'site_freq_10users.pkl'), 'wb') as site_freq_10users_pkl:
    pickle.dump(site_freq_10users, site_freq_10users_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 
                       'site_freq_150users.pkl'), 'wb') as site_freq_150users_pkl:
    pickle.dump(site_freq_150users, site_freq_150users_pkl, protocol=2)

MemoryError: 

**Чисто для подстраховки проверим, что число столбцов в разреженных матрицах `X_sparse_10users` и `X_sparse_150users` равно ранее посчитанным числам уникальных сайтов для 10 и 150 пользователей соответственно.**

In [84]:
assert X_sparse_10users.shape[1] == len(site_freq_10users)

In [85]:
assert X_sparse_150users.shape[1] == len(site_freq_150users)

## Пути улучшения
В этом проекте свобода творчества на каждом шаге, а 7 неделя проекта посвящена общему описанию (`html`, `ipynb` или `pdf`) и взаимному оцениванию проектов. Что еще можно добавить по первой части проекта:
-  можно обработать исходные данные по 3000 пользователей; обучать на такой выборке модели лучше при наличии доступа к хорошим мощностям (можно арендовать инстанс Amazon EC2, как именно, описано [тут](https://habrahabr.ru/post/280562/)). Хотя далее в курсе мы познакомимся с алгоритмами, способными обучаться на больших выборках при малых вычислительных потребностях;
- Помимо явного создания разреженного формата можно еще составить выборки с помощью `CountVectorizer`, `TfidfVectorizer` и т.п. Поскольку данные по сути могут быть описаны как последовательности, то можно вычислять n-граммы сайтов. Работает все это или нет, мы будем проверять в [соревновании](https://inclass.kaggle.com/c/identify-me-if-you-can4) Kaggle Inclass (желающие могут начать уже сейчас).

На следующей неделе мы еще немного поготовим данные и потестируем первые гипотезы, связанные с нашими наблюдениями. 